In [ ]:
# Elasticsearchインスタンスの生成

from elasticsearch import Elasticsearch

es = Elasticsearch()

In [ ]:
# リスト 3.3.2 search関数による検索
# ----------------------------------------------------------------------------------
# 目的:
#   ・Pythonクライアント(elasticsearch-py)から Elasticsearch に対して全文検索（match）を実行し、
#     レスポンス（ヒット件数・スコア・_source など）を取得・整形表示する最小例を示す。
#
# 理論メモ（挙動は変えない／解説のみ）:
#   ・match は「全文検索」クエリであり、フィールドが text 型のときアナライザでトークン化・正規化
#     された語彙を対象に、BM25（TF, IDF, 長さ正規化）でスコアリングされる。
#     - "Tanaka" は標準アナライザで "tanaka" に正規化され、"My Name Is Tanaka" に一致する。
#   ・「完全一致（生文字列の等価）」を求める場合は、text ではなく keyword サブフィールドに対して
#     term クエリ（例: "title.keyword"）を使う。語順重視は match_phrase、複数フィールドは multi_match。
#   ・検索結果は "hits": { "total": {...}, "hits": [ { "_score", "_source", ... }, ... ] } の構造。
#     - デフォルト size は 10。全件数を厳密に得たい場合は track_total_hits=true を指定する。
#   ・Elasticsearch 8.x の python クライアントでは search の推奨引数が "query" だが、
#     7.x では "body"（本コード）の形が一般的。環境に合わせてどちらかを使う（下に参考例をコメント）。
#   ・直前にドキュメント投入を行った直後は refresh 前でヒットしない場合がある。検証では
#     PUT 側に ?refresh=wait_for を付けるか、indices.refresh を明示する。
# ----------------------------------------------------------------------------------

# 検索用JSONの設定
# - 「title フィールドに 'Tanaka' を語として含む文書」を BM25 でスコアリングして取得する。
# - 複数語のとき AND/OR の挙動は operator で調整可能（例: {"query":"foo bar","operator":"and"}）。
body = {"query": {"match": {"title": "Tanaka"}}}

# （参考：Elasticsearch 8.x の推奨形。7.x 互換のため本スニペットでは未使用）
# res = es.search(index="names", query={"match": {"title": "Tanaka"}}, size=10, track_total_hits=True)

# 検索実行
# - index: 対象インデックス名。未指定での _search は全インデックス対象になり誤爆しやすいので原則明示。
# - body: 7.x 系互換の検索ボディ。8.x では query=... の使用が推奨。
res = es.search(index="names", body=body)

# 結果表示
# - ensure_ascii=False により日本語等のマルチバイトを \u エスケープせず可読表示。
# - 実務では res["hits"]["hits"] を走査して _source を取り出す処理に切り出すとよい。
import json

print(json.dumps(res, indent=2, ensure_ascii=False))

# （発展: 用途別のクエリ例）
# - 完全一致: {"query":{"term":{"title.keyword":"My Name Is Tanaka"}}}
# - フレーズ検索: {"query":{"match_phrase":{"title":{"query":"My Name"}}}}
# - タイトル重み付け + 本文も検索:
#   {"query":{"multi_match":{"query":"Tanaka","fields":["title^2","content"]}}}
# - 厳密な件数:  search(..., track_total_hits=True)
# - ソート:  {"sort":[{"_score":"desc"},{"_id":"asc"}]}  など明示的に指定する